<a target="_blank" href="https://colab.research.google.com/github/yasstake/rusty-bot/blob/main/manual/market.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Market クラス解説

Marketクラスは、取引所で扱われる通貨ペアーの１組合わせをモデリングしたクラスです。
現在はBinanceの USDBUSDのSPOT取引しか扱えませんが、比較的容易に拡張できるようにつくっていますので順次拡張していきます。

## Marketクラスメソッド

### マーケットオブジェクトの生成(`open`)

`open`メソッドでMaketオブジェクトを開きます。内部的にキャッシュされているので２回目からはキャッシュを返します。そのため複数回呼び出してOKです。

今後SPOT以外にも対応した場合、パラメータが変更になる可能性があります。
```
binance = Market.open(
    'BN',               # Binanceは 'BN'
    'BTCBUSD'
)
```

### データのダウンロード(`download`)

指定した日付あらあとのデータをダウンロードしてDBへ保存します。
BTCBUSDしか対応していないため効果はありませんが、`open`で開いたマーケットオブジェクト全てに対してダウンロード処理をします。

```
Market.download(
    5   #ダウンロード日数（N日前を指定)
    )
```


## Marketオブジェクトメソッド

Marketインスタンスが`binance`オブジェクトに保存されている前提で説明します。

###　情報表示　

####　`_repr_html_()`

jupyuter note上でオブジェクトを入力してリターンすると内部的に`_repr_html_()`がよばれて内容が分かりやすく表示されます。
```
binance
```

#### DBのパス表示 `file_path`

DBは巨大になりますので、不要になったら`file_path`プロパティで表示される場所を確認して手動で消してください。

```
binance.file_name
```

### OHLCV取得（pd.DataFrame形式)

時刻はμsで指定する。開始時刻に0を指定するとDBの最初から、終了に0を指定するとDBの最後まで（現在時刻に一番ちかいときまで）を指定したことになる。

```
binance.ohlcv(
    start_timestamp_us,     #開始時刻（μs）
    end_timestamp_us,       #終了時刻（μs)
    60                      #足の幅（秒）
    )
```

pd.DataFrame 形式でOHLCVデータが取得できます。この機能だけつかっても便利に使えます。
なお、Tickデータから任意の足を再生成しているので初回キャッシュされていないときはかなり時間がかかります。




## 以下動かしながら説明

### Google Collab向けパッケージインストール

In [ ]:
# Google Clab用です。必要に応じてコメントアウトしてください
! pip install --upgrade pip
! pip install numpy
! pip install pandas
! pip uninstall -y panel
! pip install --upgrade bokeh >= 3

# https://github.com/yasstake/rusty-bot/releases/
# （環境にあわせて修正お願いします。404エラーが出た場合は新しいリリースになっているので上記URLを確認してください）
# Linux/Google Collab用
!  pip install https://github.com/yasstake/rusty-bot/releases/download/release-0.2.0a/rbot-0.2.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl


### 以後Marktライブラリの動作

In [ ]:
# 必要ライブラリのimport
import rbot
from rbot import Market
from rbot import DAYS_BEFORE

In [ ]:
# Binance('BN')の'BTCBUSD'ペアーの取引マーケットを生成
binance = Market.open('BN', 'BTCBUSD')

# 上記でオープンしたマーケットに対し5日より新しいログをダウンロード
Market.download(5)

# Market.download(5, True)       #ディフォルトではキャッシュされているので、再ダウンロードしたい場合は強制フラグをTrueにする。Collabでは１日あたり1分ぐらいかかります。

In [ ]:
# Marketオブジェクトが管理するDBに保存されているデータ期間を表示する。
binance

In [ ]:
# データベースのDisk上の場所を表示する。不要になったら手で消してください。

binance.file_name

## OHLCVの取得

In [ ]:
# 初回は遅いです。２回目はキャッシュされているので比較的高速です
binance.ohlcv(
    DAYS_BEFORE(5),     # 5日前から
    0,                  # 最新まで
    60*60
)

短めの足では任意のサイズの足が作れるので、このOHCLVだけを利用して独自の分析を行うにも便利だと思います。
週足より長い足も作れますが、性能が出ないのとデータが揃わない点に問題があり実用できではないかもしれません。

ご活用ください。

# おまけ

## pandasで作ったDBを読み込む

以下の方法でpythonのプログラムへSQLDBから直接データをロードできる。

In [69]:
import sqlite3
import pandas as pd

file = binance.file_name

connection = sqlite3.connect(file)

df = pd.read_sql_query('select * from trades order by time_stamp limit 10000000', connection)
df['timestamp'] = pd.to_datetime(df['time_stamp'], unit='us')
df = df.set_index('timestamp', drop=True)

df = df.drop(columns='time_stamp')
df

,action,price,size,id
timestamp,,,,
2022-12-13 00:00:00.100,Sell,17212.12,0.08270,686685000
2022-12-13 00:00:00.100,Sell,17212.13,0.01260,686685001
2022-12-13 00:00:00.100,Sell,17212.13,0.30589,686685002
2022-12-13 00:00:00.100,Sell,17212.25,0.01743,686685003
2022-12-13 00:00:00.106,Buy,17211.70,0.00400,686685004
...,...,...,...,...
2022-12-16 09:17:04.954,Buy,17050.85,0.04407,696684998
2022-12-16 09:17:04.954,Buy,17050.83,0.34302,696684999
2022-12-16 09:17:04.954,Buy,17050.82,0.05244,696685000


## ohlcを作る

`resample`を使えば任意の足のohlcvが作れる。
dfに入っているデータの最後は時間きっちりではなくて、件数指定でたまたま入ってきた切れ目なので未確定足になる。

In [111]:
ohlcv = df['price'].resample('60S').ohlc()
ohlcv['volume'] = df['size'].resample('60S').sum()
ohlcv


,open,high,low,close,volume
timestamp,,,,,
2022-12-13 00:00:00,17212.12,17216.55,17208.88,17208.88,110.53964
2022-12-13 00:01:00,17209.49,17227.24,17208.84,17226.99,147.61285
2022-12-13 00:02:00,17227.23,17241.10,17225.42,17225.42,227.11544
2022-12-13 00:03:00,17226.38,17226.38,17214.26,17214.33,127.27688
2022-12-13 00:04:00,17215.36,17222.68,17208.25,17209.50,111.88474
...,...,...,...,...,...
2022-12-16 09:13:00,17041.85,17048.11,17019.24,17047.01,312.86476
2022-12-16 09:14:00,17045.76,17048.29,17021.62,17027.27,330.13224
2022-12-16 09:15:00,17028.36,17041.25,16950.24,17035.88,943.04203


## rbot.Market.ohlcvとの比較

同じ処理をする`Market.ohlcv`と比較する。

In [112]:
# 開始時刻
start_time = ohlcv.index[0]
start_time

Timestamp('2022-12-13 00:00:00', freq='60S')

In [113]:
# 終了時刻
# ohldvの終了時刻は、最後の足の開始時刻のため１本前の足の終了時刻となる（この時間までで生成したohlcvは１本少なくなる）。
end_time = ohlcv.index[-1]
end_time

Timestamp('2022-12-16 09:17:00', freq='60S')

In [131]:
# pandasの時刻は'ns'なので1000分の１のusへ変換して表示
ohlcv2 = binance.ohlcv(int(start_time.value/1000), int(end_time.value/1000), 60)
ohlcv2

,open,high,low,close,volume
timestamp,,,,,
2022-12-13 00:00:00+00:00,17212.12,17216.55,17208.88,17208.88,110.53964
2022-12-13 00:01:00+00:00,17209.49,17227.24,17208.84,17226.99,147.61285
2022-12-13 00:02:00+00:00,17227.23,17241.10,17225.42,17225.42,227.11544
2022-12-13 00:03:00+00:00,17226.38,17226.38,17214.26,17214.33,127.27688
2022-12-13 00:04:00+00:00,17215.36,17222.68,17208.25,17209.50,111.88474
...,...,...,...,...,...
2022-12-16 09:12:00+00:00,17056.28,17059.67,17024.41,17042.51,378.03635
2022-12-16 09:13:00+00:00,17041.85,17048.11,17019.24,17047.01,312.86476
2022-12-16 09:14:00+00:00,17045.76,17048.29,17021.62,17027.27,330.13224


想定どおり１本足が少なくなったがデータは一致している

## 性能比較

timeitを使って性能比較をする。

In [129]:
def panda_ohlcv():
    ohlcv = df['price'].resample('600S').ohlc()
    ohlcv['volume'] = df['size'].resample('60S').sum()

%timeit panda_ohlcv()

174 ms ± 838 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [130]:
def rust_ohlcv():
    ohlcv2 = binance.ohlcv(int(start_time.value/1000), int(end_time.value/1000), 60) 

%timeit rust_ohlcv()

61.8 ms ± 9.23 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Rust版は内部にキャッシュデータをもっていることもあって倍ぐらい早い傾向にある。